In [ ]:
#！/usr/bin/python
# -*- coding: utf-8 -*-

In [ ]:
import pyodbc
import textwrap
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
sql1 = textwrap.dedent("""
    IF OBJECT_ID('tempdb..#Diversity') IS NOT NULL
        DROP TABLE #Diversity
""")
sql2 = textwrap.dedent("""
    IF OBJECT_ID('tempdb..#csavg') IS NOT NULL
        DROP TABLE #csavg
""")
sql3 = textwrap.dedent("""
    SELECT userid
          ,LEFT(date, 8) AS [date]
          ,COUNT(DISTINCT website) AS cs
    INTO #Diversity
    FROM [data].[dbo].[health_records]
    GROUP BY userid, LEFT(date, 8)
    ORDER BY userid, LEFT(date, 8)
""")
sql4 = textwrap.dedent("""
    SELECT userid
          ,AVG(cs) AS cs_avg
    INTO #csavg
    FROM #Diversity
    GROUP BY userid
    ORDER BY userid
""")
sql5 = textwrap.dedent("""
    SELECT cs_avg AS cs
          ,COUNT(DISTINCT userid) AS cnt
          ,CONVERT(DECIMAL(4,4), COUNT(DISTINCT userid)/CONVERT(DECIMAL(5,2), (SELECT COUNT(DISTINCT userid) FROM #csavg))) AS pct
    FROM #csavg
    GROUP BY cs_avg
    ORDER BY cs
""")

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [ ]:
cursor.execute(sql1)
cursor.execute(sql2)
cursor.execute(sql3)
cursor.execute(sql4)
rows=cursor.execute(sql5).fetchall()

In [ ]:
df = pd.DataFrame( [[ij for ij in i] for i in rows] )
df.rename(columns={0:'Consideration Set', 1: 'Visitors', 2:'Percentage'}, inplace=True)

df.loc[4] = [5,0,0]

In [ ]:
trace = go.Bar(
    x = df['Consideration Set'],
    y = df['Percentage'],
    text = df['Percentage'].apply(lambda x:format(x,'.2%')),
    textposition = 'outside',
    textfont = dict(size=9,),
    width = [0.35]*5
)

data = [trace]

layout = go.Layout(
    xaxis=dict(
        title='Number of websites accessed',
        titlefont=dict(
            size=16,
        ),
        ticklen=2,
        tickfont=dict(
            size=12,
        ),
        nticks=6,
    ),
    yaxis=dict(
        title='Percentage of Visitors',
        titlefont=dict(
            size=16,
        ),
        tickfont=dict(
            size=11,
        ),
        tickformat='.0%',
        nticks=15,
        range=[0,1],
    ),
)

fig = go.Figure(data=data, layout=layout)

plotly.offline.plot(fig, filename='Diversity.html')